In [1]:
import pandas as pd

In [2]:
y2024 = pd.read_parquet('./Datasets/yellow_tripdata_2024-01.parquet')
y2025 = pd.read_parquet('./Datasets/yellow_tripdata_2025-01.parquet')
g2024 = pd.read_parquet('./Datasets/green_tripdata_2024-01.parquet')
g2025 = pd.read_parquet('./Datasets/green_tripdata_2025-01.parquet')

### check common columns

In [19]:
from functools import reduce
# รวมเป็น dict เพื่อเขียนโค้ดสั้นและขยายง่าย
dfs = {
    "y2024": y2024,
    "g2024": g2024,
    "y2025": y2025,
    "g2025": g2025,
}

# 1) หา common columns ข้ามทุก DataFrame ทีเดียว
common_cols = reduce(pd.Index.intersection, [df.columns for df in dfs.values()])

# 2) ทำตารางสรุป dtype ของแต่ละ DF บนคอลัมน์ที่ร่วมกัน
summary = pd.DataFrame({"column": common_cols})
for name, df in dfs.items():
    summary[f"dtype_{name}"] = summary["column"].map(df.dtypes)

# (ทางเลือก) เพิ่ม flag ตรวจว่า dtype ตรงกันหมดทุกชุด หรือเฉพาะคู่ปี
summary["same_dtype_y"]   = summary["dtype_y2024"].values == summary["dtype_y2025"].values
summary["same_dtype_g"]   = summary["dtype_g2024"].values == summary["dtype_g2025"].values
summary["same_dtype_all"] = summary.filter(like="dtype_").nunique(axis=1).eq(1)

# จัดให้อ่านง่าย (ถ้าต้องการ)
summary = summary.sort_values("column").reset_index(drop=True)

print(summary)

                   column dtype_y2024 dtype_g2024 dtype_y2025 dtype_g2025  \
0            DOLocationID       int32       int32       int32       int32   
1            PULocationID       int32       int32       int32       int32   
2              RatecodeID     float64     float64     float64     float64   
3                VendorID       int32       int32       int32       int32   
4    congestion_surcharge     float64     float64     float64     float64   
5                   extra     float64     float64     float64     float64   
6             fare_amount     float64     float64     float64     float64   
7   improvement_surcharge     float64     float64     float64     float64   
8                 mta_tax     float64     float64     float64     float64   
9         passenger_count     float64     float64     float64     float64   
10           payment_type       int64     float64       int64     float64   
11     store_and_fwd_flag      object      object      object      object   

In [21]:
# === คอลัมน์ที่ต่างกันระหว่าง y2024 และ g2024 ===
only_y2024 = y2024.columns.difference(g2024.columns)
only_g2024 = g2024.columns.difference(y2024.columns)

# === คอลัมน์ที่ต่างกันระหว่าง y2025 และ g2025 ===
only_y2025 = y2025.columns.difference(g2025.columns)
only_g2025 = g2025.columns.difference(y2025.columns)

# ฟังก์ชันช่วยสร้าง DataFrame พร้อม dtype
def cols_with_dtype(df, cols):
    return pd.DataFrame({
        "column": cols,
        "dtype": [df[c].dtype for c in cols]
    })

print("=== Columns only in y2024 ===")
print(cols_with_dtype(y2024, only_y2024), "\n")

print("=== Columns only in g2024 ===")
print(cols_with_dtype(g2024, only_g2024), "\n")

print("=== Columns only in y2025 ===")
print(cols_with_dtype(y2025, only_y2025), "\n")

print("=== Columns only in g2025 ===")
print(cols_with_dtype(g2025, only_g2025), "\n")


=== Columns only in y2024 ===
                  column           dtype
0            Airport_fee         float64
1  tpep_dropoff_datetime  datetime64[us]
2   tpep_pickup_datetime  datetime64[us] 

=== Columns only in g2024 ===
                  column           dtype
0              ehail_fee         float64
1  lpep_dropoff_datetime  datetime64[us]
2   lpep_pickup_datetime  datetime64[us]
3              trip_type         float64 

=== Columns only in y2025 ===
                  column           dtype
0            Airport_fee         float64
1  tpep_dropoff_datetime  datetime64[us]
2   tpep_pickup_datetime  datetime64[us] 

=== Columns only in g2025 ===
                  column           dtype
0              ehail_fee         float64
1  lpep_dropoff_datetime  datetime64[us]
2   lpep_pickup_datetime  datetime64[us]
3              trip_type         float64 

